In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
# --------------------1.数据库风格的数据合并merge / join ---------------------

In [4]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [8]:
#---------- 1.1 merge()合并，on=指定的列名作为键---------------

pd.merge(df1, df2, on='key') # 默认的inner交集，所以key=c,d,不见了

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [9]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # 键值不同，可以分别指定


,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [10]:
# how=?------inner=交集，outer=并集，left=左连接，right=右连接
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [11]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
# 多个键值进行合并
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [12]:
# 对重复列的处理
pd.merge(left, right, on='key1') # 默认是变为1_x,1_y

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [13]:
# 为重复列名指定后缀
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [14]:
# --------------1.2 也可以在index上合并，left_index=True、right_index = True-----------------
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
print(left1)
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [15]:
# 左边键值 = key ,右边根据=index
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [16]:
#---------------1.3 join() 直接表示合并索引------------------
left1.join(right1, on='key')  # 

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [17]:
# ----------------1.4 concat() 轴向连接：连接concatenation\绑定binding\堆叠stacking -------------------
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
 
pd.concat([s1, s2, s3])  # axis = 0

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [18]:
pd.concat([s1, s2, s3], axis=1) # 结果是dataFrame，并集

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [19]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [20]:
pd.concat([s1, s4], axis=1, join='inner')  # 交集

,0,1
a,0,0
b,1,1


In [21]:
pd.concat([s1, s4], axis=1) # 默认并集

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [22]:
# -----------------1.5 合并重叠的index数据：combine_first()---------------
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
print(df1)
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
print(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [23]:
df1.combine_first(df2) #用df2的数据为df1的缺失数据进行补丁操作

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [24]:
# -------------------2.Reshaping and Pivoting 重塑和轴转向----------------

In [25]:
# --------------2.1----------------
# stack：将列旋转为行
# unstack:将行旋转为列
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [26]:
# stack方法，将列旋转为行，得到series:层次化索引
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [27]:
# 对于层次化索引的Series，可以unstack:将行旋转为列，得到DataFrame
result.unstack() # 默认是最内的层索引number进行旋转

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [28]:
result.unstack('state') #指定层索引作为旋转轴

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [29]:
# --------将“长格式”旋转为“宽格式”（表格的格式）--------------
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,A,B,C,key
0,1,4,7,foo
1,2,5,8,bar
2,3,6,9,baz


In [30]:
pivoted = df.pivot('key', 'A')  # 得到层次化的列
pivoted

B              C          
A      1    2    3    1    2    3
key                              
bar  NaN  5.0  NaN  NaN  8.0  NaN
baz  NaN  NaN  6.0  NaN  NaN  9.0
foo  4.0  NaN  NaN  7.0  NaN  NaN

In [31]:
pivoted["B"]

A,1,2,3
key,,,
bar,NaN,5.0,NaN
baz,NaN,NaN,6.0
foo,4.0,NaN,NaN
